In [1]:
import pandas as pd
import os
import sys

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [21]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '1_pull_data/governance/personal_preparedness/american_community_survey/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'ACSST5Y2022.S2506-Column-Metadata.csv'


c:\Users\jespi\eagle\carb-climate-index-6\scripts\utils\file_helpers.py:78: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_data)


Saved DataFrame as 'ACSST5Y2022.S2506-Data.csv'
Saved DataFrame as 'by_county.csv'


In [3]:
# read in social service data
homeowners_insurance_data = pd.read_csv('ACSST5Y2022.S2506-Data.csv')
print(len(homeowners_insurance_data))
homeowners_insurance_data.head(5)
os.remove('ACSST5Y2022.S2506-Data.csv')

9130


C:\Users\jespi\AppData\Local\Temp\ipykernel_19204\2004692679.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  homeowners_insurance_data = pd.read_csv('ACSST5Y2022.S2506-Data.csv')


In [4]:
homeowners_insurance_data

,GEO_ID,NAME,S2506_C01_001E,S2506_C01_001M,S2506_C01_002E,S2506_C01_002M,S2506_C01_003E,S2506_C01_003M,S2506_C01_004E,S2506_C01_004M,...,S2506_C02_062M,S2506_C02_063E,S2506_C02_063M,S2506_C02_064E,S2506_C02_064M,S2506_C02_065E,S2506_C02_065M,S2506_C02_066E,S2506_C02_066M,Unnamed: 266
0,Geography,Geographic Area Name,Estimate!!Owner-occupied housing units with a ...,Margin of Error!!Owner-occupied housing units ...,Estimate!!Owner-occupied housing units with a ...,Margin of Error!!Owner-occupied housing units ...,Estimate!!Owner-occupied housing units with a ...,Margin of Error!!Owner-occupied housing units ...,Estimate!!Owner-occupied housing units with a ...,Margin of Error!!Owner-occupied housing units ...,...,Margin of Error!!Percent owner-occupied housin...,Estimate!!Percent owner-occupied housing units...,Margin of Error!!Percent owner-occupied housin...,Estimate!!Percent owner-occupied housing units...,Margin of Error!!Percent owner-occupied housin...,Estimate!!Percent owner-occupied housing units...,Margin of Error!!Percent owner-occupied housin...,Estimate!!Percent owner-occupied housing units...,Margin of Error!!Percent owner-occupied housin...,NaN
1,1400000US06001400100,Census Tract 4001; Alameda County; California,758,163,0,13,0,13,18,20,...,5.1,0.0,5.1,98.9,1.5,1.1,1.5,"10,000+",***,NaN
2,1400000US06001400200,Census Tract 4002; Alameda County; California,314,59,0,13,0,13,0,13,...,11.9,0.0,11.9,100.0,11.9,0.0,11.9,"10,000+",***,NaN
3,1400000US06001400300,Census Tract 4003; Alameda County; California,694,198,13,21,0,19,0,19,...,5.6,0.0,5.6,98.1,3.0,1.9,3.0,"10,000+",***,NaN
4,1400000US06001400400,Census Tract 4004; Alameda County; California,495,128,0,13,0,13,0,13,...,7.8,0.0,7.8,100.0,7.8,0.0,7.8,"10,000+",***,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9125,1400000US06115040902,Census Tract 409.02; Yuba County; California,3,5,0,13,0,13,3,5,...,100.0,0.0,100.0,100.0,100.0,0.0,100.0,-,**,NaN
9126,1400000US06115041001,Census Tract 410.01; Yuba County; California,808,235,0,13,9,14,156,183,...,20.4,0.0,4.8,62.7,20.6,1.7,2.9,2595,917,NaN
9127,1400000US06115041002,Census Tract 410.02; Yuba County; California,665,200,0,13,0,13,95,71,...,5.8,0.0,5.8,100.0,5.8,0.0,5.8,4775,756,NaN
9128,1400000US06115041101,Census Tract 411.01; Yuba County; California,423,191,0,13,1,3,117,76,...,4.0,40.2,29.9,57.7,29.4,0.0,9.0,1703,660,NaN


In [16]:
columns = [
            'GEO_ID',
            'S2506_C02_002E',
            'S2506_C02_003E',
            'S2506_C02_004E',
            'S2506_C02_005E',
            'S2506_C02_006E',
            'S2506_C02_007E',
            'S2506_C02_008E'
]

columns2 = ['GEO_ID', 'S2506_C02_001E', 'S2506_C01_001E']

homeowners_insurance_data_columns = homeowners_insurance_data[columns2]
homeowners_insurance_data_columns

,GEO_ID,S2506_C02_001E,S2506_C01_001E
0,Geography,Estimate!!Percent owner-occupied housing units...,Estimate!!Owner-occupied housing units with a ...
1,1400000US06001400100,758,758
2,1400000US06001400200,314,314
3,1400000US06001400300,694,694
4,1400000US06001400400,495,495
...,...,...,...
9125,1400000US06115040902,3,3
9126,1400000US06115041001,808,808
9127,1400000US06115041002,665,665
9128,1400000US06115041101,423,423


In [14]:
homeowners_insurance_data_numeric = homeowners_insurance_data.drop(columns='GEO_ID').apply(pd.to_numeric, errors='coerce')
homeowners_insurance_data_columns['sum_percent_mortgage'] = homeowners_insurance_data_numeric.sum(axis=1)
homeowners_insurance_data_columns

C:\Users\jespi\AppData\Local\Temp\ipykernel_19204\2086068960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  homeowners_insurance_data_columns['sum_percent_mortgage'] = homeowners_insurance_data_numeric.sum(axis=1)


,GEO_ID,S2506_C02_002E,S2506_C02_003E,S2506_C02_004E,S2506_C02_005E,S2506_C02_006E,S2506_C02_007E,S2506_C02_008E,sum_percent_mortgage
0,Geography,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,Estimate!!Percent owner-occupied housing units...,0.0
1,1400000US06001400100,0.0,0.0,2.4,3.7,0.0,1.2,92.7,3549456.5
2,1400000US06001400200,0.0,0.0,0.0,1.3,0.0,4.1,94.6,4647131.3
3,1400000US06001400300,1.9,0.0,0.0,0.0,2.3,4.6,91.2,4436312.9
4,1400000US06001400400,0.0,0.0,0.0,1.8,6.7,11.3,80.2,3913872.7
...,...,...,...,...,...,...,...,...,...
9125,1400000US06115040902,0.0,0.0,100.0,0.0,0.0,0.0,0.0,7764.0
9126,1400000US06115041001,0.0,1.1,19.3,43.3,31.3,1.0,4.0,1282903.3
9127,1400000US06115041002,0.0,0.0,14.3,19.1,42.0,13.5,11.1,1688257.3
9128,1400000US06115041101,0.0,0.2,27.7,70.9,0.5,0.0,0.7,981274.0


Trying county data

In [102]:
# read in social service data
homeowners_insurance_data_county = pd.read_csv('by_county.csv')
print(len(homeowners_insurance_data_county))
homeowners_insurance_data_county = homeowners_insurance_data_county.rename(columns={'Label (Grouping)':'index'})
homeowners_insurance_data_county = homeowners_insurance_data_county[1:]
homeowners_insurance_data_county.head(5)

# os.remove('by_county.csv')

160


,index,"Alameda County, California!!Estimate","Alameda County, California!!Margin of Error","Alameda County, California!!Percent","Alameda County, California!!Percent Margin of Error","Butte County, California!!Estimate","Butte County, California!!Margin of Error","Butte County, California!!Percent","Butte County, California!!Percent Margin of Error","Contra Costa County, California!!Estimate",...,"Ventura County, California!!Percent","Ventura County, California!!Percent Margin of Error","Yolo County, California!!Estimate","Yolo County, California!!Margin of Error","Yolo County, California!!Percent","Yolo County, California!!Percent Margin of Error","Yuba County, California!!Estimate","Yuba County, California!!Margin of Error","Yuba County, California!!Percent","Yuba County, California!!Percent Margin of Error"
1,Total housing units,"629,159","±1,950","629,159",(X),"90,309",±546,"90,309",(X),"426,327",...,"294,645",(X),"81,252",±635,"81,252",(X),"29,973",±234,"29,973",(X)
2,Occupied housing units,"589,180","±4,004",93.6%,±0.5,"81,353","±1,910",90.1%,±2.0,"411,560",...,94.7%,±0.9,"76,844","±1,623",94.6%,±1.7,"27,942",±908,93.2%,±2.9
3,Vacant housing units,"39,979","±3,194",6.4%,±0.5,"8,956","±1,847",9.9%,±2.0,"14,767",...,5.3%,±0.9,"4,408","±1,360",5.4%,±1.7,"2,031",±882,6.8%,±2.9
4,Homeowner vacancy rate,0.7,±0.3,(X),(X),1.4,±0.8,(X),(X),0.8,...,(X),(X),0.0,±0.5,(X),(X),0.3,±0.5,(X),(X)
5,Rental vacancy rate,6.1,±0.9,(X),(X),4.4,±2.0,(X),(X),1.9,...,(X),(X),1.5,±1.3,(X),(X),2.0,±2.8,(X),(X)


In [104]:
# Transpose the DataFrame
#homeowners_insurance_data_county_transpose = homeowners_insurance_data_county.transpose()
homeowners_insurance_data_county_transpose = homeowners_insurance_data_county.set_index('index').T.reset_index()

# Set the first row as the column headers
#new_header = homeowners_insurance_data_county_transpose.iloc[0]
#homeowners_insurance_data_county_transpose = homeowners_insurance_data_county_transpose[1:]
#homeowners_insurance_data_county_transpose.columns = new_header

homeowners_insurance_data_county_transpose

index,index,Total housing units,Occupied housing units,Vacant housing units,Homeowner vacancy rate,Rental vacancy rate,UNITS IN STRUCTURE,Total housing units,"1-unit, detached","1-unit, attached",...,No rent paid,GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI),Occupied units paying rent (excluding units where GRAPI cannot be computed),Less than 15.0 percent,15.0 to 19.9 percent,20.0 to 24.9 percent,25.0 to 29.9 percent,30.0 to 34.9 percent,35.0 percent or more,Not computed
0,"Alameda County, California!!Estimate","629,159","589,180","39,979",0.7,6.1,NaN,"629,159","322,776","56,494",...,"7,107",NaN,"256,368","31,030","33,291","32,166","29,095","24,710","106,076","10,942"
1,"Alameda County, California!!Margin of Error","±1,950","±4,004","±3,194",±0.3,±0.9,NaN,"±1,950","±5,587","±3,099",...,"±1,108",NaN,"±6,109","±2,659","±2,904","±2,634","±2,962","±3,414","±5,453","±1,524"
2,"Alameda County, California!!Percent","629,159",93.6%,6.4%,(X),(X),NaN,"629,159",51.3%,9.0%,...,(X),NaN,"256,368",12.1%,13.0%,12.5%,11.3%,9.6%,41.4%,(X)
3,"Alameda County, California!!Percent Margin of ...",(X),±0.5,±0.5,(X),(X),NaN,(X),±0.9,±0.5,...,(X),NaN,(X),±1.0,±1.1,±1.0,±1.1,±1.3,±1.9,(X)
4,"Butte County, California!!Estimate","90,309","81,353","8,956",1.4,4.4,NaN,"90,309","55,771","2,372",...,"1,445",NaN,"31,556","2,788","2,808","4,164","4,304","1,951","15,541","2,254"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,"Yolo County, California!!Percent Margin of Error",(X),±1.7,±1.7,(X),(X),NaN,(X),±2.7,±1.2,...,(X),NaN,(X),±2.2,±3.5,±3.3,±3.8,±2.5,±5.7,(X)
164,"Yuba County, California!!Estimate","29,973","27,942","2,031",0.3,2.0,NaN,"29,973","19,634","2,001",...,N,NaN,"10,480",525,"1,985","1,302",776,"1,715","4,177","1,141"
165,"Yuba County, California!!Margin of Error",±234,±908,±882,±0.5,±2.8,NaN,±234,"±1,505",±828,...,N,NaN,"±1,540",±390,±903,±690,±423,±917,±978,±885
166,"Yuba County, California!!Percent","29,973",93.2%,6.8%,(X),(X),NaN,"29,973",65.5%,6.7%,...,(X),NaN,"10,480",5.0%,18.9%,12.4%,7.4%,16.4%,39.9%,(X)


In [105]:
homeowners_insurance_data_county_transpose.columns

Index(['index', '    Total housing units', '        Occupied housing units',
       '        Vacant housing units', '        Homeowner vacancy rate',
       '        Rental vacancy rate', 'UNITS IN STRUCTURE',
       '    Total housing units', '        1-unit, detached',
       '        1-unit, attached',
       ...
       '        No rent paid',
       'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME (GRAPI)',
       '    Occupied units paying rent (excluding units where GRAPI cannot be computed)',
       '        Less than 15.0 percent', '        15.0 to 19.9 percent',
       '        20.0 to 24.9 percent', '        25.0 to 29.9 percent',
       '        30.0 to 34.9 percent', '        35.0 percent or more',
       '        Not computed'],
      dtype='object', name='index', length=160)

In [125]:
# Define the columns you want to select
columns = [
    'index',
    '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Housing units with a mortgage']
# Select the specified columns
homeowners_insurance_county_columns = homeowners_insurance_data_county_transpose[columns]
homeowners_insurance_county_columns = homeowners_insurance_county_columns.rename(columns={'MORTGAGE STATUS':'morgage_status', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Housing units with a mortgage':'housing_units_with_morgage', 'index':'county'})
# Display the selected columns
homeowners_insurance_county_columns.head()

index,county,housing_units_with_morgage
0,"Alameda County, California!!Estimate","221,103"
1,"Alameda County, California!!Margin of Error","±5,945"
2,"Alameda County, California!!Percent",68.7%
3,"Alameda County, California!!Percent Margin of ...",±1.2
4,"Butte County, California!!Estimate","26,714"


In [126]:
homeowners_insurance_county_columns.columns

Index(['county', 'housing_units_with_morgage'], dtype='object', name='index')

In [127]:
homeowners_insurance_county_columns

index,county,housing_units_with_morgage
0,"Alameda County, California!!Estimate","221,103"
1,"Alameda County, California!!Margin of Error","±5,945"
2,"Alameda County, California!!Percent",68.7%
3,"Alameda County, California!!Percent Margin of ...",±1.2
4,"Butte County, California!!Estimate","26,714"
...,...,...
163,"Yolo County, California!!Percent Margin of Error",±3.1
164,"Yuba County, California!!Estimate","11,009"
165,"Yuba County, California!!Margin of Error","±1,442"
166,"Yuba County, California!!Percent",67.5%


In [139]:
filtered_county_morgage = homeowners_insurance_county_columns[~homeowners_insurance_county_columns['county'].str.contains('Margin|Estimate')]

print(len(filtered_county_morgage))
filtered_county_morgage.head()

42


index,county,housing_units_with_morgage
2,"Alameda County, California!!Percent",68.7%
6,"Butte County, California!!Percent",56.2%
10,"Contra Costa County, California!!Percent",72.2%
14,"El Dorado County, California!!Percent",68.2%
18,"Fresno County, California!!Percent",65.4%


In [141]:
# Filter and remove " County, California!!Percent" from 'county' column
filtered_county_mortgage = filtered_county_morgage[~filtered_county_morgage['county'].str.contains('Margin')]
filtered_county_mortgage['county'] = filtered_county_mortgage['county'].str.replace(' County, California!!Percent', '')

print(len(filtered_county_mortgage))
filtered_county_mortgage.head(50)

42


index,county,housing_units_with_morgage
2,Alameda,68.7%
6,Butte,56.2%
10,Contra Costa,72.2%
14,El Dorado,68.2%
18,Fresno,65.4%
22,Humboldt,57.2%
26,Imperial,61.6%
30,Kern,67.1%
34,Kings,66.2%
38,Lake,52.6%
